In [2]:
import os
import random
import shutil
from pathlib import Path
import json
import math

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, roc_curve
import pandas as pd

## Nature set(ground truth + val set)

In [2]:
generator_names = ["adm", "bgan", "glide", "midj", "sd_14", "sd_15", "vqdm", "wukong"]

with open("class.json", "r", encoding="utf-8") as f:
    data = json.load(f)
classes_idx = data["1k_idx"]
classes_names = data["21k_idx"]


def map1k21k(idx_1k):
    names_21k = []
    idx_21k = []
    names_1k = []
    with open("../Data/GenImage/imagenet21k_wordnet_ids.txt", "r", encoding="utf-8") as f:
        for idx, line in enumerate(f):
            line = line.strip()
            idx_21k.append(idx + 1)
            names_21k.append(line)
    with open("../Data/GenImage/map21k21k.txt", "r", encoding="utf-8") as f:
        for idx, line in enumerate(f):
            line = line.strip()
            names_1k.append(line)
            assert int(line) != 9206 and int(line) != 15028
    print(len(names_21k), len(names_1k))
    corresponding_names = [names_21k[int(names_1k[int(idx_1k[i])])] for i in range(len(idx_1k))]
    return corresponding_names


# try:
#     if len(my_list) == 0:
#         print("列表为空")
#     else:
#         print("列表不为空")
# except NameError:
#     print("变量 my_list 不存在")
# except TypeError:
#     print("变量 my_list 不是一个可以使用 len() 的对象（可能是 None）")



def verify_train_nature(top_path):
    nature_path = Path(top_path + "/train/nature")
    nature_paths = []
    for idx, class_id in enumerate(classes_names):
        nature_pattern = f"{class_id}_*"
        nature_files = list(nature_path.glob(nature_pattern))
        # real_images = list(Path(nature_dir).glob("*"))
        if not nature_files:
            print(f"Error: Class{class_id} has no pictures in {nature_path}.")
        nature_paths.append(nature_files)
    return nature_paths


In [3]:
gt_paths_1 = verify_train_nature("F:/Data/imagenet_adm")

Error: Classn02085782 has no pictures in F:\Data\imagenet_adm\train\nature.
Error: Classn02087046 has no pictures in F:\Data\imagenet_adm\train\nature.
Error: Classn02089078 has no pictures in F:\Data\imagenet_adm\train\nature.
Error: Classn02089973 has no pictures in F:\Data\imagenet_adm\train\nature.
Error: Classn02091635 has no pictures in F:\Data\imagenet_adm\train\nature.
Error: Classn02095889 has no pictures in F:\Data\imagenet_adm\train\nature.
Error: Classn02102973 has no pictures in F:\Data\imagenet_adm\train\nature.
Error: Classn02110627 has no pictures in F:\Data\imagenet_adm\train\nature.
Error: Classn02113978 has no pictures in F:\Data\imagenet_adm\train\nature.
Error: Classn02606052 has no pictures in F:\Data\imagenet_adm\train\nature.
Error: Classn03197337 has no pictures in F:\Data\imagenet_adm\train\nature.
Error: Classn03498962 has no pictures in F:\Data\imagenet_adm\train\nature.
Error: Classn03920288 has no pictures in F:\Data\imagenet_adm\train\nature.
Error: Class

In [6]:
gt_paths = [gt_paths_1]

In [7]:
for generator in generator_names[1:]:
    gt_path = verify_train_nature(f"F:/Data/imagenet_{generator}")
    gt_paths.append(gt_path)

Error: Classn03013580 has no pictures in F:\Data\imagenet_bgan\train\nature.
Error: Classn03013580 has no pictures in F:\Data\imagenet_glide\train\nature.
Error: Classn03013580 has no pictures in F:\Data\imagenet_midj\train\nature.
Error: Classn03013580 has no pictures in F:\Data\imagenet_sd_14\train\nature.
Error: Classn01688243 has no pictures in F:\Data\imagenet_sd_15\train\nature.
Error: Classn01744401 has no pictures in F:\Data\imagenet_sd_15\train\nature.
Error: Classn02085782 has no pictures in F:\Data\imagenet_sd_15\train\nature.
Error: Classn02087046 has no pictures in F:\Data\imagenet_sd_15\train\nature.
Error: Classn02089078 has no pictures in F:\Data\imagenet_sd_15\train\nature.
Error: Classn02089867 has no pictures in F:\Data\imagenet_sd_15\train\nature.
Error: Classn02089973 has no pictures in F:\Data\imagenet_sd_15\train\nature.
Error: Classn02091635 has no pictures in F:\Data\imagenet_sd_15\train\nature.
Error: Classn02095314 has no pictures in F:\Data\imagenet_sd_15\tr

In [13]:
gt_paths_merged = [sum(sublists, []) for sublists in zip(*gt_paths)]

In [ ]:
with open("nature_paths_origine.json", "w", encoding="utf-8") as f:
    json.dump([[p.__str__() for p in group] for group in gt_paths_merged], f, indent=2)

# with open("nested_paths.json", "r", encoding="utf-8") as f:
#     loaded = json.load(f)
#     gt_paths = [[Path(p) for p in group] for group in loaded]

In [17]:
minL = len(gt_paths_merged[0])
maxL = len(gt_paths_merged[0])
for i in range(len(gt_paths_merged)):
    if len(gt_paths_merged[i]) < minL:
        minL = len(gt_paths_merged[i])
        print(i)
    if len(gt_paths_merged[i]) > maxL:
        maxL = len(gt_paths_merged[i])
print(minL, maxL)

43
62
152
165
850
0 1300


In [19]:
print(len(gt_paths_merged[43]), len(gt_paths_merged[62]), len(gt_paths_merged[152]), len(gt_paths_merged[165]), len(gt_paths_merged[850]))

1117 1071 772 732 0


In [18]:
classes_names[850]

'n03013580'

In [4]:
classes_idx = classes_idx[:850] + classes_idx[851:]
classes_names = classes_names[:850] + classes_names[851:]

In [22]:
a = gt_paths_merged.pop(850)
print(a, len(gt_paths_merged))

[] 999


In [21]:
random_seed = 2025
random.seed(random_seed)

In [24]:
nature_gt_paths = []
nature_val_paths = []
for i in range(len(gt_paths_merged)):
    nature_gt_paths.append([])
    nature_val_paths.append([])
    L = list(range(len(gt_paths_merged[i])))
    image_idx = random.sample(L, 550)
    nature_gt_paths[i].extend([gt_paths_merged[i][idx] for idx in image_idx[:500]])
    nature_val_paths[i].extend([gt_paths_merged[i][idx] for idx in image_idx[500:]])

In [25]:
for i in range(len(nature_gt_paths)):
    assert len(nature_gt_paths[i]) == 500
    assert len(nature_val_paths[i]) == 50

In [29]:
d_paths = {"gt": [[str(p) for p in group] for group in nature_gt_paths] , "val": [[str(p) for p in group] for group in nature_val_paths]}
with open("nature_paths.json", "w", encoding="utf-8") as f:
    json.dump(d_paths, f, indent=2)

In [31]:
import clip
from transformers import AutoImageProcessor, AutoModel, AutoProcessor, AutoModelForImageClassification

d:\IPTP\Stage_3A\Stage\Code\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
class Dinov2Dataset(Dataset):
    def __init__(self, image_paths):
        self.image_paths = image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        try:
            image = Image.open(path).convert("RGB")
            return image, str(path)
        except Exception as e:
            print(f"Failure open image because of {e}")
            return None


def dinov2_collate_fn(batch):
    batch = [item for item in batch if item is not None]
    if not batch:
        return None, None
    images, paths = zip(*batch)
    return list(images), paths

In [33]:
def dinov2_encode(image_paths, batch_size = 64, model_name='facebook/dinov2-with-registers-base', device='cuda'):
    processor = AutoImageProcessor.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    dataset = Dinov2Dataset(image_paths)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=dinov2_collate_fn)
    embeddings = []

    for images, paths in tqdm(dataloader, desc="Extracting cls tokens"):
        if images is None:
                continue
        # processor expects a list of PIL images
        inputs = processor(images=images, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            cls_token = outputs.last_hidden_state[:, 0, :]  # remove CLS
            embeddings.append(cls_token.detach().cpu())

    embedding_all = torch.cat(embeddings, dim=0)
    
    return embedding_all


In [34]:
for i in range(len(classes_idx)):
    save_path_gt = f"../Data/Dataset/dinov2/gt/{classes_idx[i]}.pt"
    save_path_nval = f"../Data/Dataset/dinov2/nval/{classes_idx[i]}.pt"
    os.makedirs(os.path.dirname(save_path_gt), exist_ok=True)
    os.makedirs(os.path.dirname(save_path_nval), exist_ok=True)

    gt_embeddings = dinov2_encode(nature_gt_paths[i], batch_size = 128)
    torch.save(gt_embeddings, save_path_gt)
    nval_embeddings = dinov2_encode(nature_val_paths[i], batch_size = 64)
    torch.save(nval_embeddings, save_path_nval)
    

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Extracting cls tokens:   0%|          | 0/4 [00:00<?, ?it/s]d:\IPTP\Stage_3A\Stage\Code\.venv\lib\site-packages\transformers\integrations\sdpa_attention.py:54: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Extracting cls tokens: 100%|██████████| 1/1 [00:02<00:00,  2.72s/it]


## AI test set

In [38]:
ai_paths = []

In [59]:
def verify_ai(
    top_path,
):
    ai_path = Path(top_path + "/val/ai")
    # nature_path = Path(top_path + "/val/nature")
    ai_paths = []
    # nature_paths = []
    for class_id in classes_idx:
        # ai_pattern = f"{int(class_id)}_adm_*.png"
        # ai_pattern = f"{class_id}_biggan_*.png"
        # ai_pattern = f"*_{class_id}_glide_*.png"
        # ai_pattern = f"{int(class_id)}_midjourney_*.png"
        # ai_pattern = f"{class_id}_sdv4_*.png"
        # ai_pattern = f"{class_id}_sdv5_*.png"
        # ai_pattern = f"*_{class_id}_vqdm_*.png"
        ai_pattern = f"{int(class_id)}_wukong_*.png"
        ai_files = list(ai_path.glob(ai_pattern))
        try:
            assert len(ai_files) == 6
        except AssertionError as e:
            print(f"AI files in {ai_path}/{class_id} not 6, but {len(ai_files)}")
        ai_paths.append(ai_files)
    return ai_paths

In [ ]:
ai_paths.append(verify_ai("F:/Data/imagenet_adm"))
ai_paths.append(verify_ai("F:/Data/imagenet_bgan"))
ai_paths.append(verify_ai("F:/Data/imagenet_glide"))
ai_paths.append(verify_ai("F:/Data/imagenet_midj"))
ai_paths.append(verify_ai("F:/Data/imagenet_sd_14"))
ai_paths.append(verify_ai("F:/Data/imagenet_sd_15"))
ai_paths.append(verify_ai("F:/Data/imagenet_vqdm"))
ai_paths.append(verify_ai("F:/Data/imagenet_wukong"))

In [61]:
ai_paths_merged = [sum(sublists, []) for sublists in zip(*ai_paths)]

In [62]:
with open("ai_paths.json", "w", encoding="utf-8") as f:
    json.dump([[p.__str__() for p in group] for group in ai_paths_merged], f, indent=2)

In [63]:
for i in range(len(classes_idx)):
    save_path_ai = f"../Data/Dataset/dinov2/aval/{classes_idx[i]}.pt"
    # save_path_nval = f"../Data/Dataset/dinov2/nval/{classes_idx[i]}.pt"
    os.makedirs(os.path.dirname(save_path_ai), exist_ok=True)
    # os.makedirs(os.path.dirname(save_path_nval), exist_ok=True)

    ai_embeddings = dinov2_encode(ai_paths_merged[i], batch_size = 64)
    torch.save(ai_embeddings, save_path_ai)
    # nval_embeddings = dinov2_encode(nature_val_paths[i], batch_size = 64)
    # torch.save(nval_embeddings, save_path_nval)

Extracting cls tokens: 100%|██████████| 1/1 [00:02<00:00,  2.46s/it]


## Mahalanobis with cls token

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
generator_names = ["adm", "bgan", "glide", "midj", "sd_14", "sd_15", "vqdm", "wukong"]
with open("class.json", "r", encoding="utf-8") as f:
    data = json.load(f)
classes_idx = data["1k_idx"]
classes_names = data["21k_idx"]
classes_idx = classes_idx[:850] + classes_idx[851:]
classes_names = classes_names[:850] + classes_names[851:]

In [67]:
def gt_compute(embeddings, eps=1e-3):
    mean = embeddings.mean(dim=0, keepdim=True)
    X = embeddings - mean
    cov = X.T @ X / (embeddings.size(0) - 1)
    cov += eps * torch.eye(cov.size(0), device=embeddings.device)
    return mean, cov

def mahalanobis_distance(x, mean, cov):
    x = x.to(torch.float32).view(-1)
    mean = mean.to(torch.float32).view(-1)
    delta = x - mean
    cov = cov.to(torch.float32)

    try:
        sol = torch.linalg.solve(cov, delta.unsqueeze(1))  # [D, 1]
        dist_squared = delta @ sol.squeeze()
        if dist_squared < 0:
            print("Warning: distance squared < 0", dist_squared.item())
            dist_squared = torch.clamp(dist_squared, min=0.0)
        dist = torch.sqrt(dist_squared)
        return dist
    except RuntimeError as e:
        print("Runtime error in Mahalanobis:", e)
        return torch.tensor(float("nan"), device=x.device)

In [ ]:
def mahalanobis_detector():
    ### real = 0, fake = 1
    # os.makedirs(os.path.dirname(save_path), exist_ok=True)
    data = {"CLASS":[], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95":[]}
    # for cls in classes_idx:
    for cls in tqdm(classes_idx, desc="Detection computing"):
        gt_all = torch.load(f"../Data/Dataset/dinov2/gt/{cls}.pt", weights_only = True)
        nval_all = torch.load(f"../Data/Dataset/dinov2/nval/{cls}.pt", weights_only = True)
        aval_all = torch.load(f"../Data/Dataset/dinov2/aval/{cls}.pt", weights_only = True)
        gt_tokens = gt_all[:240].to(device)
        gt_mean, gt_cov = gt_compute(gt_tokens)
        gt_mean, gt_cov = gt_mean.to(device), gt_cov.to(device)

        for i in range(len(generator_names)):
        # for i in tqdm(range(len(generator_names)), desc="Detection computing"):
            data["CLASS"].append(cls)
            data["GENERATOR"].append(generator_names[i])
            if i<5:
                nval_tokens = nval_all[i*6:(i+1)*6]
                aval_tokens = aval_all[i*6:(i+1)*6]
            elif i==5:
                nval_tokens = nval_all[i*6:i*6+8]
                aval_tokens = aval_all[i*6:i*6+8]
            elif i>5:
                nval_tokens = nval_all[(i-1)*6+8:i*6+8]
                aval_tokens = aval_all[(i-1)*6+8:i*6+8]
            test_tokens = torch.cat([nval_tokens, aval_tokens], dim=0)
            labels = np.concatenate((np.zeros(nval_tokens.shape[0]), np.ones(aval_tokens.shape[0])))
            test_scores = []
            for sample in test_tokens.to(device):
                distance = mahalanobis_distance(sample, gt_mean, gt_cov)
                test_scores.append(distance.cpu())
            scores = np.array(test_scores)
            fpr, tpr, thresholds = roc_curve(labels, scores)
            idx = np.where(tpr >= 0.95)[0][0]
            fpr_95 = fpr[idx]

            # distances = np.sqrt((1 - tpr) ** 2 + fpr**2)
            # best_threshold = thresholds[np.argmin(distances)]
            # print("Best threshold(ROC):", best_threshold)

            roc_auc = roc_auc_score(labels, scores)
            precision, recall, thresholds = precision_recall_curve(labels, scores)
            pr_auc = auc(recall, precision)

            data["AUROC"].append(roc_auc)
            data["AUPRC"].append(pr_auc)
            data["FPR95"].append(fpr_95)


    df = pd.DataFrame(data)
    df.to_csv("dinov2_cls_m_result.csv", index=False)
    print(f"dinov2_cls auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")
    return df

In [ ]:
m_df = mahalanobis_detector()

Detection computing: 100%|██████████| 999/999 [04:44<00:00,  3.51it/s]

dinov2_cls auroc: 0.7388873248248248, auprc: 0.7307166117495979, fpr95: 0.44887074574574576


## RPO Sparse

In [4]:
import torch
import os
import json
import math
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
from transformers import AutoImageProcessor, AutoModel, AutoProcessor, AutoModelForImageClassification
import torch.nn.functional as F
from sklearn.random_projection import SparseRandomProjection, GaussianRandomProjection
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, roc_curve
from scipy.special import softmax

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
generator_names = ["adm", "bgan", "glide", "midj", "sd_14", "sd_15", "vqdm", "wukong"]
with open("class.json", "r", encoding="utf-8") as f:
    data = json.load(f)
classes_idx = data["1k_idx"]
classes_names = data["21k_idx"]

classes_idx = classes_idx[:850] + classes_idx[851:]
classes_names = classes_names[:850] + classes_names[851:]

In [5]:
def mad(x, dim=-1, keepdim=False):
    """
    计算 median absolute deviation (MAD)
    MAD = median(|x - median(x)|)
    """
    med = x.median(dim=dim, keepdim=True)[0]
    mad = (x - med).abs().median(dim=dim, keepdim=keepdim)[0]
    # 避免除零
    mad = torch.clamp(mad, min=1e-6)
    return mad

class SparseRPO:
    def __init__(self, D, dtype, M=1000, s=None, device='cuda', seed=None):
        """
        D: 原始维度
        M: 随机投影个数
        s: 稀疏度参数，默认 sqrt(D)
        """
        self.D = D
        self.M = M
        self.device = device
        self.s = s or int(math.sqrt(D))
        if seed is not None:
            torch.manual_seed(seed)
        
        self.U = self._generate_sparse_projection_matrix(dtype)

    def _generate_sparse_projection_matrix(self, dtype):
        D, M, s = self.D, self.M, self.s
        prob_nonzero = 1.0 / s

        # 初始化全零矩阵
        U = torch.zeros(M, D, device=self.device, dtype=dtype)

        # 生成均匀随机数矩阵
        rand_vals = torch.rand(M, D, device=self.device)

        pos_mask = rand_vals < (1/(2*s))
        neg_mask = (rand_vals >= (1/(2*s))) & (rand_vals < (1/s))

        val = math.sqrt(s)
        U[pos_mask] = val
        U[neg_mask] = -val

        # 注意：这里没有归一化，sklearn也是这样实现的

        return U

    def fit(self, X):
        """
        计算每个投影方向上的 MED 和 MAD
        X: (B, D)
        """
        X = X.to(self.device)
        proj = torch.matmul(self.U, X.T)  # (M, B)
        self.med = proj.median(dim=1)[0]  # (M,)
        self.mad = mad(proj, dim=1)        # (M,)

    def score(self, x):
        """
        计算批量测试样本的RPO分数
        x: (N, D)
        """
        x = x.to(self.device)
        proj_x = torch.matmul(self.U, x.T)  # (M, N)
        med = self.med.unsqueeze(1)          # (M, 1)
        mad = self.mad.unsqueeze(1)          # (M, 1)

        score = torch.abs(proj_x - med) / mad  # (M, N)
        rpo_score, _ = score.max(dim=0)        # (N,)

        return rpo_score

In [6]:
def rpo_detector(M=1000, seed=2025):
    data = {"CLASS":[], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95":[]}
    for cls in tqdm(classes_idx, desc="Detection computing"):
        gt_all = torch.load(f"../Data/Dataset/dinov2/gt/{cls}.pt", weights_only = True)
        nval_all = torch.load(f"../Data/Dataset/dinov2/nval/{cls}.pt", weights_only = True)
        aval_all = torch.load(f"../Data/Dataset/dinov2/aval/{cls}.pt", weights_only = True)
        gt_tokens = gt_all[:240].to(device)
        datatype = gt_tokens.dtype
        dim = gt_tokens.shape[1]
        rpo = SparseRPO(D=dim, dtype = datatype, M = M, device = device, seed = seed)
        rpo.fit(gt_tokens)
        
        for i in range(len(generator_names)):
        # for i in tqdm(range(len(generator_names)), desc="Detection computing"):
            data["CLASS"].append(cls)
            data["GENERATOR"].append(generator_names[i])
            if i<5:
                nval_tokens = nval_all[i*6:(i+1)*6]
                aval_tokens = aval_all[i*6:(i+1)*6]
            elif i==5:
                nval_tokens = nval_all[i*6:i*6+8]
                aval_tokens = aval_all[i*6:i*6+8]
            elif i>5:
                nval_tokens = nval_all[(i-1)*6+8:i*6+8]
                aval_tokens = aval_all[(i-1)*6+8:i*6+8]
            test_tokens = torch.cat([nval_tokens, aval_tokens], dim=0)
            labels = np.concatenate((np.zeros(nval_tokens.shape[0]), np.ones(aval_tokens.shape[0])))
            test_tokens = test_tokens.to(device)
            scores = rpo.score(test_tokens).cpu()
            scores = scores.numpy()

            fpr, tpr, thresholds = roc_curve(labels, scores)
            idx = np.where(tpr >= 0.95)[0][0]
            fpr_95 = fpr[idx]
            roc_auc = roc_auc_score(labels, scores)
            precision, recall, thresholds = precision_recall_curve(labels, scores)
            pr_auc = auc(recall, precision)

            data["AUROC"].append(roc_auc)
            data["AUPRC"].append(pr_auc)
            data["FPR95"].append(fpr_95)

    df = pd.DataFrame(data)
    df.to_csv("dinov2_cls_rpo2_result.csv", index=False)
    print(f"dinov2_cls auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")
    return df

In [14]:
rpo2_df = rpo_detector()

Detection computing: 100%|██████████| 999/999 [00:33<00:00, 29.78it/s]

dinov2_cls auroc: 0.677318377405183, auprc: 0.6844689245586236, fpr95: 0.5548152318985652


## CNNSpot

In [77]:
adm_array = np.load("CNNDetection-master/cnnresult_adm.npy")
adm_array[1]

array([4.68072609e-15, 1.55764121e-06, 3.94324928e-08, ...,
       1.00000000e+00, 6.16390025e-04, 7.53998563e-07], shape=(12000,))

In [81]:
def analysis(array_path):
    array = np.load(array_path)
    labels = array[0]
    pred = array[1]
    fpr, tpr, thresholds = roc_curve(labels, pred)
    idx = np.where(tpr >= 0.95)[0][0]
    fpr_95 = fpr[idx]

    distances = np.sqrt((1 - tpr) ** 2 + fpr**2)
    best_threshold = thresholds[np.argmin(distances)]
    print("Best threshold(ROC):", best_threshold)

    roc_auc = roc_auc_score(labels, pred)
    precision, recall, thresholds = precision_recall_curve(labels, pred)
    pr_auc = auc(recall, precision)
    print(roc_auc, pr_auc, fpr_95)
    return roc_auc, pr_auc, fpr_95
    

In [83]:
r1,p1,f1 = analysis("CNNDetection-master/cnnresult_adm.npy")

Best threshold(ROC): 1.8014501620200463e-05
0.7364656527777778 0.7123961176033554 0.7213333333333334


In [84]:
r2,p2,f2 = analysis("CNNDetection-master/cnnresult_bgan.npy")

Best threshold(ROC): 0.009483017027378082
0.9247847361111111 0.9002123184864556 0.24466666666666667


In [85]:
r3,p3,f3 = analysis("CNNDetection-master/cnnresult_glide.npy")

Best threshold(ROC): 3.047243581022485e-06
0.705534625 0.6632481497482541 0.7153333333333334


In [86]:
r4,p4,f4 = analysis("CNNDetection-master/cnnresult_midj.npy")

Best threshold(ROC): 1.264304728465504e-06
0.5643275833333333 0.5602116157286022 0.9256666666666666


In [87]:
r5,p5,f5 = analysis("CNNDetection-master/cnnresult_sd_14.npy")

Best threshold(ROC): 1.1308032981105498e-06
0.606037 0.5693771165774026 0.8656666666666667


In [88]:
r6,p6,f6 = analysis("CNNDetection-master/cnnresult_sd_15.npy")

Best threshold(ROC): 9.126443956120056e-07
0.60820890625 0.5731092047877862 0.86025


In [89]:
r7,p7,f7 = analysis("CNNDetection-master/cnnresult_vqdm.npy")

Best threshold(ROC): 2.718274117796682e-06
0.6582188333333334 0.6205565248947923 0.8138333333333333


In [90]:
r8,p8,f8 = analysis("CNNDetection-master/cnnresult_wukong.npy")

Best threshold(ROC): 1.7776693539417465e-07
0.5541938611111111 0.5289734443212323 0.913


In [91]:
data = {"GENERATOR": generator_names, "AUROC": [r1,r2,r3,r4,r5,r6,r7,r8], "AUPRC": [p1,p2,p3,p4,p5,p6,p7,p8], "FPR95": [f1,f2,f3,f4,f5,f6,f7,f8]}
df = pd.DataFrame(data)
df.to_csv("CNNSpot_result.csv", index = False)

In [92]:
df

,GENERATOR,AUROC,AUPRC,FPR95
0,adm,0.736466,0.712396,0.721333
1,bgan,0.924785,0.900212,0.244667
2,glide,0.705535,0.663248,0.715333
3,midj,0.564328,0.560212,0.925667
4,sd_14,0.606037,0.569377,0.865667
5,sd_15,0.608209,0.573109,0.860250
6,vqdm,0.658219,0.620557,0.813833
7,wukong,0.554194,0.528973,0.913000


## RESNET from DIRE

In [6]:
def analyze_DIRE(generator):
    ai_array = np.load(f"DiffusionForensics/DIRE_{generator}_ai.npy")
    nature_array = np.load(f"DiffusionForensics/DIRE_{generator}_nature.npy")
    probs = np.hstack((nature_array, ai_array))
    labels = np.concatenate((np.zeros(len(nature_array)), np.ones(len(ai_array))))
    fpr, tpr, thresholds = roc_curve(labels, probs)
    idx = np.where(tpr >= 0.95)[0][0]
    fpr_95 = fpr[idx]

    distances = np.sqrt((1 - tpr) ** 2 + fpr**2)
    best_threshold = thresholds[np.argmin(distances)]
    print("Best threshold(ROC):", best_threshold)

    roc_auc = roc_auc_score(labels, probs)
    precision, recall, thresholds = precision_recall_curve(labels, probs)
    pr_auc = auc(recall, precision)
    print(roc_auc, pr_auc, fpr_95)
    return roc_auc, pr_auc, fpr_95

In [7]:
data = {"GENERATOR":[], "AUROC":[], "AUPRC":[], "FPR95":[]}
for generator in generator_names:
    r,p,f = analyze_DIRE(generator)
    data["GENERATOR"].append(generator)
    data["AUROC"].append(r)
    data["AUPRC"].append(p)
    data["FPR95"].append(f)
df = pd.DataFrame(data)
df.to_csv("DIRE_result.csv", index = False)


Best threshold(ROC): 9.56907215670821e-17
0.5312362222222222 0.5197698998157685 0.9365
Best threshold(ROC): 5.339503846528556e-15
0.685177375 0.6310851585099351 0.8193333333333334
Best threshold(ROC): 1.7498194559814508e-16
0.5421131527777778 0.5330483026654933 0.9655
Best threshold(ROC): 3.730572320341089e-17
0.5131300694444445 0.504718363373807 0.9698333333333333
Best threshold(ROC): 5.11320703032919e-18
0.4747256388888889 0.47446831863868744 0.9593333333333334
Best threshold(ROC): 6.5722783574369884e-18
0.4769408828125 0.47282163633488483 0.955625
Best threshold(ROC): 1.7918090920504212e-14
0.5964097638888889 0.6167859686731915 0.9613333333333334
Best threshold(ROC): 2.1080356698911232e-18
0.3722946944444444 0.4086551363198635 0.9906666666666667


## AEROBLADE

In [2]:
generator_names = ["adm", "bgan", "glide", "midj", "sd_14", "sd_15", "vqdm", "wukong"]

In [10]:
def aeroblade_analysis(path):
    data = {"GENERATOR":[], "AUROC":[], "AUPRC":[], "FPR95":[]}
    for generator in generator_names:
        data["GENERATOR"].append(generator)

        ndf = pd.read_csv(path + f"/{generator}_nature.csv")
        adf = pd.read_csv(path + f"/{generator}_ai.csv")
        ndistance = ndf[ndf['repo_id']=='max']['distance'].values
        adistance = adf[adf['repo_id']=='max']['distance'].values
        distances = np.hstack((ndistance,adistance))
        labels = np.concatenate((np.zeros(len(ndistance)), np.ones(len(adistance))))
        fpr, tpr, thresholds = roc_curve(labels, distances)
        idx = np.where(tpr >= 0.95)[0][0]
        fpr_95 = fpr[idx]

        dist = np.sqrt((1 - tpr) ** 2 + fpr**2)
        best_threshold = thresholds[np.argmin(dist)]
        print("Best threshold(ROC):", best_threshold)

        roc_auc = roc_auc_score(labels, distances)
        precision, recall, thresholds = precision_recall_curve(labels, distances)
        pr_auc = auc(recall, precision)
        print(roc_auc, pr_auc, fpr_95)

        data["AUROC"].append(roc_auc)
        data["AUPRC"].append(pr_auc)
        data["FPR95"].append(fpr_95)

    df = pd.DataFrame(data)
    df.to_csv("AEROBLADE_result.csv", index = False)
    return data


In [9]:
a_df = aeroblade_analysis("aeroblade_outputs")

Best threshold(ROC): -0.0300445556640625
0.6395962083333334 0.6754059904455245 0.9428333333333333
Best threshold(ROC): -0.0275726318359375
0.803197111111111 0.7656517014047645 0.5923333333333334
Best threshold(ROC): -0.0216064453125
0.9174768888888889 0.9221742006357854 0.44583333333333336
Best threshold(ROC): -0.026123046875
0.8092533194444445 0.810955513791197 0.6895
Best threshold(ROC): -0.03277587890625
0.5602262777777778 0.5493688126420764 0.9003333333333333
Best threshold(ROC): -0.03277587890625
0.5630705859375 0.5501652500474099 0.89725
Best threshold(ROC): -0.034149169921875
0.4912112638888889 0.520222676466287 0.9826666666666667
Best threshold(ROC): -0.031494140625
0.6239498611111111 0.6072865351088843 0.8705


## Comparison

In [11]:
generator_names = ["adm", "bgan", "glide", "midj", "sd_14", "sd_15", "vqdm", "wukong"]

In [12]:
def read_result(file_path,save_path):
    df = pd.read_csv(file_path)
    result1 = df.groupby("GENERATOR", as_index = False)[["AUROC", "AUPRC", "FPR95"]].mean()
    result2 = df[["AUROC", "AUPRC", "FPR95"]].mean()

    new_row = pd.DataFrame({
        "GENERATOR": ["average"],
        "AUROC": [result2["AUROC"]],
        "AUPRC": [result2["AUPRC"]],
        "FPR95": [result2["FPR95"]]
    })

    final_df = pd.concat([result1, new_row], ignore_index=True)
    final_df.to_csv(save_path)
    return final_df


In [13]:
df1 = read_result("dinov2cls_mahalanobis_genimage.csv", "method1.csv")
df2 = read_result("dinov2cls_rpo2_genimage.csv", "method2.csv")

## DRCT

In [6]:
GenImage_LIST = ['sd_14', 'sd_15', 'midj', 'adm', 'wukong', 'glide', 'vqdm', 'bgan']

In [9]:
def DRCT_analysis():
    data = {"GENERATOR":[], "AUROC":[], "AUPRC":[], "FPR95":[]}
    for i, name in enumerate(GenImage_LIST):
        data["GENERATOR"].append(name)
        probs = np.load(f"DRCT_results/GenImage_metrics_convB_probs_{i+1}.npy")
        labels = np.concatenate((np.zeros(int(0.5* len(probs))), np.ones(int(0.5* len(probs)))))
        fpr, tpr, thresholds = roc_curve(labels, probs)
        idx = np.where(tpr >= 0.95)[0][0]
        fpr_95 = fpr[idx]

        distances = np.sqrt((1 - tpr) ** 2 + fpr**2)
        best_threshold = thresholds[np.argmin(distances)]
        print("Best threshold(ROC):", best_threshold)

        roc_auc = roc_auc_score(labels, probs)
        precision, recall, thresholds = precision_recall_curve(labels, probs)
        pr_auc = auc(recall, precision)
        data["AUROC"].append(roc_auc)
        data["AUPRC"].append(pr_auc)
        data["FPR95"].append(fpr_95)
    df = pd.DataFrame(data)
    df.to_csv("final results/DRCT_retults.csv", index = False)
    return data


In [10]:
res = DRCT_analysis()

Best threshold(ROC): 0.9731518
Best threshold(ROC): 0.8920366
Best threshold(ROC): 0.013261795
Best threshold(ROC): 0.00037574768
Best threshold(ROC): 0.9122301
Best threshold(ROC): 0.0007749796
Best threshold(ROC): 0.00040203333
Best threshold(ROC): 0.0010126233
